<a href="https://colab.research.google.com/github/kozanitis/tutorials/blob/master/tensorFlow_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello world!!!!')

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

### Check GPU access


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU not found!!!')
else:
  print('GPU successfully found!!!')

## A warmup: Compare CPU vs GPU execution in TensorFlow:

Small exercise: Create a conv layer over a random image and manually place the
operation on a CPU and on a GPU. Run both elements 10 times


In [ ]:
%tensorflow_version 2.x

import timeit

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

## Now Let's create a Neural Network and train it on the MNIST dataset

MNIST is a popular collection of images of handwritten digits. 

### Step 1: Download the dataset
TensorFlow can automatically retrieve this dataset

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ttl_images = 10
images = x_test[:ttl_images]
labels = y_test[:ttl_images]
num_row = 2
num_col = 5
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(ttl_images):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[i]))
plt.tight_layout()
plt.show()



### Step 2. Build a Machine Learning model.
Data images are expected to be 28x28 grayscale pixels and labels (0-9) are represented as a vector of 10 classes. For each image, the model will probabilities an image belongs to each class.

We are going to create a single 3-layer model.
Layer 1: Input Image of 28x28 units
Layer 2: A Dense layer of 128 units
Layer 3: A Dense layer of 10 units. Layer 3 will use softmax as an activation function to convert the scores of each class to a probability.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2), #to avoid overfitting
  tf.keras.layers.Dense(10, activation='softmax')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Step 3: train the model


In [ ]:
with tf.device('/device:GPU:0'):
  model.fit(x_train, y_train, epochs=5)

### Step 4. Evaluate the model

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

### Step 5. Run predictions

In [ ]:
import numpy as np
predictions = model(x_test[:5])

np.argmax(predictions, axis=1)